# Text Classification Pipeline
## Packages

In [ ]:
import os
from termcolor import colored
from forte.data.readers import ClassificationDatasetReader
from fortex.huggingface import ZeroShotClassifier
from forte.pipeline import Pipeline
from fortex.nltk import NLTKSentenceSegmenter
from ft.onto.base_ontology import Sentence


## Background
This notebook tutorial is derived from [a classification example](https://github.com/asyml/forte/tree/master/examples/classification).
Given a table-like csv file with some columns of input text data and one column of the label, we set up a text classification pipeline below. This tutorial is also an excellent example of wrapping external library classes/methods into `PipelineComponent`.


## Inference Workflow

### Pipeline
* [Pipeline setup](https://github.com/asyml/forte/blob/master/examples/classification/bank_customer_intent.py#L123)

* The pipeline has one reader `ClassificationDatasetReader` and two processor
`NLTKSentenceSegmenter` and `ZeroShotClassifier`. 


### Reader
* [ClassificationDatasetReader](https://github.com/asyml/forte/blob/7dc6e6c7d62d9a4126bdfc5ca02d15be3ffd61ca/forte/data/readers/classification_reader.py#L26)
 * `set_up()`: It checks whether the configuration is correct. For example, `skip_k_starting_lines` should be larger than 0. Otherwise, it doesn't make sense. It also converts different table data at the label column to a digit.
 * `_collect()`: read rows from csv file and returns an iterator that yields line id and line data.
 * `_cache_key_function()`: use the line id as the cache key. 
 * `_parse_pack()`: parse data from the iterator returned by `_collect` and load it in the ``DataPack``



### Processor
In this example, we want to classify data sentence by sentence so we wrapped `nltk.PunktSentenceTokenizer` in [NLTKSentenceSegmenter](https://github.com/asyml/forte-wrappers/blob/80cfe19926c0596edd13985581e8ca01a7be86ad/src/nltk/fortex/nltk/nltk_processors.py#L247) to segment sentences. 

* `_process()`: split ``DataPack`` text into sentence spans.



Then need a model to do classification. We wrap `transformers.pipeline` in 
[Huggingface ZeroShotClassifier](https://github.com/asyml/forte-wrappers/blob/main/src/huggingface/fortex/huggingface/zero_shot_classifier.py).

* `_process()`: running classifier over ``DataPack`` data and writing the prediction results back to the ``DataPack``.

`ZeroShotClassifier` and `NLTKSentenceSegmenter` both inherit from `PackProcessor` as it processes one `DataPack` at a time. Suppose if we process one `MultiPack` at a time, we must inherit `MultiPackProcessor` instead. 

In [2]:

csv_path = os.path.abspath(
            os.path.join("data_samples", "amazon_review_polarity_csv/sample.csv")
        )  # notebook should be running from project root folder
pl = Pipeline()

# initialize labels
class_names = ["negative", "positive"]
index2class = dict(enumerate(class_names))
pl.set_reader(
    ClassificationDatasetReader(), config={"index2class": index2class}
)
pl.add(NLTKSentenceSegmenter())
pl.add(ZeroShotClassifier(), config={"candidate_labels": class_names})
pl.initialize()


for pack in pl.process_dataset(csv_path):
    for sent in pack.get(Sentence):
        sent_text = sent.text
        print(colored("Sentence:", "red"), sent_text, "\n")
        print(colored("Prediction:", "blue"), sent.classification)

[nltk_data] Downloading package punkt to /home/murphy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sentence: One of the best game music soundtracks - for a game I didn't really play
Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. 

Prediction: {'positive': 0.954, 'negative': 0.0054}
Sentence: There is an incredible mix of fun, epic, and emotional songs. 

Prediction: {'positive': 0.0115, 'negative': 0.0001}
Sentence: Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs in my other video game soundtracks. 

Prediction: {'negative': 0.0002, 'positive': 0.0001}
Sentence: I must admit that one of the songs (Life-A Distant Promise) has brought tears to my eyes on many occasions.My one complaint about this soundtrack is that they use guitar fretting effects in many of the songs, which I find distracting. 

Prediction: {'positive': 0.0365, 'negative': 0.0028}
Sentence